In [6]:
from elasticsearch import Elasticsearch
import time
from datetime import timedelta, datetime
from lorem_text import lorem

es = Elasticsearch("http://127.0.0.1:9200")

print(f"Connected to ElasticSearch cluster `{es.info().body['cluster_name']}`")

Connected to ElasticSearch cluster `docker-cluster`


# Consultas a la base de datos con 1 millón de registros y todos los campos indexados

En esta sección se revisará la operación de búsqueda para la colección de 1 millón de documentos ya indexados

In [7]:
idx = "extracto_cuenta_1m"

## Consulta 1: Todos los datos

Se realizará una búsqueda sobre todos los campos en la colección

In [6]:
query = {
    "match_all" : {}
}

start_time = time.time()
results = es.search(index=idx, query=query)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '1993-09-22 12:40:22', 'agencia': 'Oruro', 'monto': 197861.14, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 1822809.72, 'nota': 'ducimus officia eum praesentium nesciunt dolorum assumenda distinctio repellendus facilis inventore ad quod'}

{'fecha': '1951-05-28 22:45:09', 'agencia': 'Beni', 'monto': 333582.65, 'descripcion': 'Pago recibo de luz', 'saldo': 3022748.45, 'nota': 'aliquid hic blanditiis in magni impedit fugit accusamus sint'}

{'fecha': '1951-08-31 01:49:14', 'agencia': 'Chuquisaca', 'monto': 243308.44, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 1315661.01, 'nota': 'corporis repellendus'}

{'fecha': '2018-02-02 12:06:52', 'agencia': 'Oruro', 'monto': 188642.12, 'descripcion': 'Pago en línea - Servicio de internet', 'saldo': 3208976.13, 'nota': 'quis nobis quae officiis'}

{'fecha': '2001-03-27 05:18:31', 'agencia': 'Pando', 'monto': 379968.28, 'descripcion': 'Pago recibo de luz', 'saldo': 806113.97, 'nota': 'illo harum quis itaque aliquam doloribus vitae temporibus qu

## Consulta 2: Fecha y agencia

Se realizará una búsqueda sobre los campos fecha y agencia. Siendo los parámetros de entrada una fecha inicial, una fecha final y una agencia.

In [56]:
def busqueda_fecha_agencia(start_date, end_date, agencia):
    from_date_str = start_date + " 00:00:00"
    to_date_str = end_date + " 23:59:59"
    
    query = {
            "bool": {
                "must": [
                    { "match": { "agencia": f"{agencia}" } },
                    {"range" : {
                        "fecha": { "gte" : f"{from_date_str}", "lte" : f"{to_date_str}"}}
                    }
                ]
            }
    }

    return es.search(index=idx, query=query)

In [14]:
start_date = "2019-01-31" 
end_date = "2019-12-31"
agencia = "Cochabamba"

start_time = time.time()
results = busqueda_fecha_agencia(start_date, end_date, agencia)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '2019-07-04 15:16:28', 'agencia': 'Cochabamba', 'monto': 133559.25, 'descripcion': 'Debito Cta por ACH', 'saldo': 2217674.49, 'nota': 'quae possimus nihil quasi nobis sed voluptatum beatae eius suscipit expedita similique'}

{'fecha': '2019-03-16 05:36:53', 'agencia': 'Cochabamba', 'monto': 200424.83, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 7578881.93, 'nota': 'provident repellat vero quas tenetur nemo ab vitae ullam'}

{'fecha': '2019-03-15 03:27:10', 'agencia': 'Cochabamba', 'monto': 84705.22, 'descripcion': 'Pago de nómina', 'saldo': 828603.3, 'nota': 'eum rerum fugit eos'}

{'fecha': '2019-09-19 04:56:24', 'agencia': 'Cochabamba', 'monto': 84075.4, 'descripcion': 'Consignación en efectivo', 'saldo': 8636216.21, 'nota': 'architecto doloribus illo nesciunt quia est fugiat ut aperiam'}

{'fecha': '2019-12-20 14:55:25', 'agencia': 'Cochabamba', 'monto': 71924.54, 'descripcion': 'Comisión de mantenimiento', 'saldo': 7171625.59, 'nota': 'magni aut nam prov

## Consulta 3: Rango de saldo, dos opciones de agencias y descripción

Se realizará una búsqueda sobre los campos saldo, agencia (OR 2 opciones) y descripción. Los parámetros de entrada son un rango de saldos, dos opciones de agencias, y una descripción.

In [57]:
def busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion):
    query = {
            "bool": {
                "must": [
                    {
                        "bool": {
                            "should": [
                                {"match": {"agencia": f"{agencia1}"}},
                                {"match": {"agencia": f"{agencia2}"}}
                            ]
                        }
                    },
                    {
                        "bool": {
                            "must": [
                                {
                                    "range" : {
                                        "monto": { "gte" : saldo_min, "lte": saldo_max}
                                    }
                                }
                            ]
                        }
                    },
                    {
                        "match": {"descripcion": f"{descripcion}"}
                    }
                ]
            }
    }

    return es.search(index=idx, query=query)

In [21]:
saldo_min = 162342
saldo_max = 382304513
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = "Retiro en efectivo"

start_time = time.time()
results = busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '1998-08-10 00:44:49', 'agencia': 'Santa Cruz', 'monto': 309718.61, 'descripcion': 'Retiro en efectivo', 'saldo': 914647.58, 'nota': 'voluptatibus illum voluptas odit ex incidunt'}

{'fecha': '1986-10-01 09:10:00', 'agencia': 'Santa Cruz', 'monto': 357624.23, 'descripcion': 'Retiro en efectivo', 'saldo': 8658740.86, 'nota': 'harum illo tempore officiis sapiente officia dignissimos natus alias atque'}

{'fecha': '1994-07-19 23:13:52', 'agencia': 'Santa Cruz', 'monto': 370078.84, 'descripcion': 'Retiro en efectivo', 'saldo': 7041823.23, 'nota': 'quia quisquam similique'}

{'fecha': '1979-04-04 11:58:10', 'agencia': 'Santa Cruz', 'monto': 295316.64, 'descripcion': 'Retiro en efectivo', 'saldo': 4278890.78, 'nota': 'quae enim labore nemo'}

{'fecha': '1975-06-07 23:41:33', 'agencia': 'Santa Cruz', 'monto': 387581.17, 'descripcion': 'Retiro en efectivo', 'saldo': 1225706.16, 'nota': 'porro quibusdam reiciendis aperiam amet'}

{'fecha': '1959-08-19 15:19:16', 'agencia': 'Santa Cruz

## Agregación 1: Suma de saldos de una agencia

Se calculará la suma de los saldos de los datos agrupados de una agencia en específico.

In [22]:
def suma_saldo_agencia(agencia):
    aggs = {
            f"suma {agencia}" : {
                    "sum" : {
                        "field" : "saldo",
                    } 
            }
        }

    query = {
        "match": {"agencia": f"{agencia}"}
    }



    return es.search(index=idx, query=query, aggs=aggs)

In [23]:
agencia = "Beni"

start_time = time.time()
results = suma_saldo_agencia(agencia)
end_time = time.time()


print("="*100)
print(results['hits'])
print("="*100)
print(results['aggregations'])
print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 2.0775523, 'hits': [{'_index': 'extracto_cuenta_1m', '_id': 'gK2XXoQBoKrfaqsDWq-H', '_score': 2.0775523, '_source': {'fecha': '1951-05-28 22:45:09', 'agencia': 'Beni', 'monto': 333582.65, 'descripcion': 'Pago recibo de luz', 'saldo': 3022748.45, 'nota': 'aliquid hic blanditiis in magni impedit fugit accusamus sint'}}, {'_index': 'extracto_cuenta_1m', '_id': 'j62XXoQBoKrfaqsDWq-H', '_score': 2.0775523, '_source': {'fecha': '1966-12-30 01:28:56', 'agencia': 'Beni', 'monto': 271624.59, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 3606574.85, 'nota': 'quibusdam'}}, {'_index': 'extracto_cuenta_1m', '_id': 'nq2XXoQBoKrfaqsDWq-H', '_score': 2.0775523, '_source': {'fecha': '2003-11-02 02:24:14', 'agencia': 'Beni', 'monto': 165310.57, 'descripcion': 'Pago recibo de agua', 'saldo': 157449.56, 'nota': 'maxime corporis odit quis non molestias tempore esse dicta ipsa mollitia reiciendis atque'}}, {'_index': 'extracto_cuent

## Agregación 2: Promedio de montos por agencia

In [60]:
def promedio_montos_agencia():
    aggs = {
            "promedio":
            {
                "terms":
                {
                    "field": "agencia"
                },
                "aggs":
                {
                    "promedio":
                    {
                        "avg": {
                            "field": "monto"
                        }
                    }
                }
            }
            
    }


    return es.search(index=idx, aggs=aggs)

In [62]:
start_time = time.time()
results = promedio_montos_agencia()
end_time = time.time()


print("="*100)
print(results['hits'])
print("="*100)
print(results['aggregations'])
print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'extracto_cuenta_1m', '_id': 'f62XXoQBoKrfaqsDWq-H', '_score': 1.0, '_source': {'fecha': '1993-09-22 12:40:22', 'agencia': 'Oruro', 'monto': 197861.14, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 1822809.72, 'nota': 'ducimus officia eum praesentium nesciunt dolorum assumenda distinctio repellendus facilis inventore ad quod'}}, {'_index': 'extracto_cuenta_1m', '_id': 'gK2XXoQBoKrfaqsDWq-H', '_score': 1.0, '_source': {'fecha': '1951-05-28 22:45:09', 'agencia': 'Beni', 'monto': 333582.65, 'descripcion': 'Pago recibo de luz', 'saldo': 3022748.45, 'nota': 'aliquid hic blanditiis in magni impedit fugit accusamus sint'}}, {'_index': 'extracto_cuenta_1m', '_id': 'ga2XXoQBoKrfaqsDWq-H', '_score': 1.0, '_source': {'fecha': '1951-08-31 01:49:14', 'agencia': 'Chuquisaca', 'monto': 243308.44, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 1315661.01, 'nota': 'corporis repellendus'}}, {'_index': 'extracto_cuenta_1m', '_id'

**Nota:** Elasticsearch no recomienda las agregaciones por campos del tipo texto, en su lugar recomienda utilizar campos del tipo keyword.

![](../imagenes-de-soporte/error_aggs_elastic.png)

# Consultas a la base de datos con 10 millones de registros y todos los campos indexados

En esta sección se revisará la operación de búsqueda para la colección de 10 millones de documentos indexados.

In [63]:
idx = "extracto_cuenta_10m"

## Consulta 1: Todos los datos

Se realizará una búsqueda sobre todos los campos en la colección

In [64]:
query = {
    "match_all" : {}
}

start_time = time.time()
results = es.search(index=idx, query=query)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '1999-06-18 17:01:14', 'agencia': 'La Paz', 'monto': 298916.5, 'descripcion': 'Pago de cheque compensado', 'saldo': 4401608.54, 'nota': 'aperiam reprehenderit delectus accusantium quam in ratione deleniti perspiciatis'}

{'fecha': '1988-06-28 16:09:15', 'agencia': 'La Paz', 'monto': 61522.6, 'descripcion': 'Pago recibo de agua', 'saldo': 3105995.8, 'nota': 'eaque vel accusamus quam sit consequatur enim illo laborum iure cumque repellat'}

{'fecha': '2019-04-24 07:27:04', 'agencia': 'Tarija', 'monto': 357643.79, 'descripcion': 'Retiro en efectivo', 'saldo': 8027374.93, 'nota': 'illum iusto ipsa autem repudiandae minima nam voluptates temporibus libero'}

{'fecha': '1970-05-09 17:53:11', 'agencia': 'Beni', 'monto': 225384.23, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 7513100.25, 'nota': 'assumenda reiciendis provident fugit'}

{'fecha': '2009-04-04 07:54:43', 'agencia': 'Chuquisaca', 'monto': 93061.78, 'descripcion': 'Retiro en efectivo', 'saldo': 6005020.11, 'nota

## Consulta 2: Fecha y agencia

Se realizará una búsqueda sobre los campos fecha y agencia. Siendo los parámetros de entrada una fecha inicial, una fecha final y una agencia.

In [65]:
start_date = "2019-01-31" 
end_date = "2019-12-31"
agencia = "Cochabamba"

start_time = time.time()
results = busqueda_fecha_agencia(start_date, end_date, agencia)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '2019-12-20 14:55:25', 'agencia': 'Cochabamba', 'monto': 71924.54, 'descripcion': 'Comisión de mantenimiento', 'saldo': 7171625.59, 'nota': 'magni aut nam provident'}

{'fecha': '2019-12-16 00:04:32', 'agencia': 'Cochabamba', 'monto': 273700.97, 'descripcion': 'Pago recibo de gas', 'saldo': 4218202.71, 'nota': 'rerum molestias obcaecati dolorum eveniet repellendus hic praesentium odit autem alias temporibus'}

{'fecha': '2019-07-04 09:30:37', 'agencia': 'Cochabamba', 'monto': 267080.69, 'descripcion': 'Ingreso en efectivo', 'saldo': 3530247.67, 'nota': 'nulla rerum explicabo unde qui iure ipsum consectetur saepe minus amet dicta recusandae'}

{'fecha': '2019-11-15 13:33:54', 'agencia': 'Cochabamba', 'monto': 370397.39, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7569939.65, 'nota': 'delectus a consequatur voluptatibus alias et'}

{'fecha': '2019-02-12 07:37:22', 'agencia': 'Cochabamba', 'monto': 362102.95, 'descripcion': 'Pago en interés', 'saldo': 885465.53, 'nota': 'a s

## Consulta 3: Rango de saldo, dos opciones de agencias y descripción

Se realizará una búsqueda sobre los campos saldo, agencia (OR 2 opciones) y descripción. Los parámetros de entrada son un rango de saldos, dos opciones de agencias, y una descripción.

In [66]:
saldo_min = 162342
saldo_max = 382304513
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = "Retiro en efectivo"

start_time = time.time()
results = busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '1971-09-05 03:22:25', 'agencia': 'La Paz', 'monto': 387034.16, 'descripcion': 'Retiro en efectivo', 'saldo': 3873302.17, 'nota': 'hic maiores sed ullam odio molestias nulla a'}

{'fecha': '2007-09-19 07:26:56', 'agencia': 'La Paz', 'monto': 305979.29, 'descripcion': 'Retiro en efectivo', 'saldo': 8334222.29, 'nota': 'iusto mollitia necessitatibus reprehenderit eaque corrupti libero consequuntur nemo excepturi ex facilis aspernatur'}

{'fecha': '1983-11-18 12:28:01', 'agencia': 'La Paz', 'monto': 287408.17, 'descripcion': 'Retiro en efectivo', 'saldo': 2481328.47, 'nota': 'incidunt veniam delectus hic reprehenderit ex quidem quaerat eligendi'}

{'fecha': '1977-04-02 05:06:15', 'agencia': 'La Paz', 'monto': 261529.35, 'descripcion': 'Retiro en efectivo', 'saldo': 8216476.33, 'nota': 'perspiciatis deleniti totam fugiat numquam iusto delectus adipisci rem dicta culpa pariatur'}

{'fecha': '1981-12-31 05:23:17', 'agencia': 'La Paz', 'monto': 256193.53, 'descripcion': 'Retiro en e

## Agregación 1: Suma de saldos de una agencia

Se calculará la suma de los saldos de los datos agrupados de una agencia en específico.

In [67]:
agencia = "Beni"

start_time = time.time()
results = suma_saldo_agencia(agencia)
end_time = time.time()


print("="*100)
print(results['hits'])
print("="*100)
print(results['aggregations'])
print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 2.0802577, 'hits': [{'_index': 'extracto_cuenta_10m', '_id': 'og3JXoQBoKrfaqsD8LFO', '_score': 2.0802577, '_source': {'fecha': '1970-05-09 17:53:11', 'agencia': 'Beni', 'monto': 225384.23, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 7513100.25, 'nota': 'assumenda reiciendis provident fugit'}}, {'_index': 'extracto_cuenta_10m', '_id': 'rw3JXoQBoKrfaqsD8LFO', '_score': 2.0802577, '_source': {'fecha': '1982-06-16 20:04:50', 'agencia': 'Beni', 'monto': 230684.84, 'descripcion': 'Cobro seguro', 'saldo': 8272273.59, 'nota': 'at eligendi ullam enim'}}, {'_index': 'extracto_cuenta_10m', '_id': 'zA3JXoQBoKrfaqsD8LFO', '_score': 2.0802577, '_source': {'fecha': '1988-11-10 14:42:03', 'agencia': 'Beni', 'monto': 219408.7, 'descripcion': 'Deposito a cuenta', 'saldo': 4807462.76, 'nota': 'cupiditate eligendi inventore dolorum ullam id laboriosam'}}, {'_index': 'extracto_cuenta_10m', '_id': '0g3JXoQBoKrfaqsD8LFO', '_score': 2.0802

## Agregación 2: Promedio de montos por agencia

In [68]:
start_time = time.time()
results = promedio_montos_agencia()
end_time = time.time()


print("="*100)
print(results['hits'])
print("="*100)
print(results['aggregations'])
print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'extracto_cuenta_10m', '_id': 'nw3JXoQBoKrfaqsD8LFO', '_score': 1.0, '_source': {'fecha': '1999-06-18 17:01:14', 'agencia': 'La Paz', 'monto': 298916.5, 'descripcion': 'Pago de cheque compensado', 'saldo': 4401608.54, 'nota': 'aperiam reprehenderit delectus accusantium quam in ratione deleniti perspiciatis'}}, {'_index': 'extracto_cuenta_10m', '_id': 'oA3JXoQBoKrfaqsD8LFO', '_score': 1.0, '_source': {'fecha': '1988-06-28 16:09:15', 'agencia': 'La Paz', 'monto': 61522.6, 'descripcion': 'Pago recibo de agua', 'saldo': 3105995.8, 'nota': 'eaque vel accusamus quam sit consequatur enim illo laborum iure cumque repellat'}}, {'_index': 'extracto_cuenta_10m', '_id': 'oQ3JXoQBoKrfaqsD8LFO', '_score': 1.0, '_source': {'fecha': '2019-04-24 07:27:04', 'agencia': 'Tarija', 'monto': 357643.79, 'descripcion': 'Retiro en efectivo', 'saldo': 8027374.93, 'nota': 'illum iusto ipsa autem repudiandae minima nam voluptates 

# Consultas a la base de datos con 30 millones de registros y todos los campos indexados

En esta sección se revisará la operación de búsqueda para la colección de 30 millones de documentos indexados.

In [69]:
idx = "extracto_cuenta_30m"

## Consulta 1: Todos los datos

Se realizará una búsqueda sobre todos los campos en la colección

In [70]:
query = {
    "match_all" : {}
}

start_time = time.time()
results = es.search(index=idx, query=query)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '1985-01-08 00:43:31', 'agencia': 'Chuquisaca', 'monto': 224529.15, 'descripcion': 'Ingreso en efectivo', 'saldo': 3536179.16, 'nota': 'distinctio iste'}

{'fecha': '1972-10-10 07:50:07', 'agencia': 'Oruro', 'monto': 314217.46, 'descripcion': 'Pago de nómina', 'saldo': 7217329.69, 'nota': 'quidem'}

{'fecha': '1955-06-10 17:59:28', 'agencia': 'Beni', 'monto': 182999.8, 'descripcion': 'Retención IVA', 'saldo': 365443.46, 'nota': 'quod voluptate quidem eius corporis blanditiis eveniet voluptatibus mollitia aut'}

{'fecha': '1957-02-14 06:52:33', 'agencia': 'Santa Cruz', 'monto': 115836.12, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 4240546.69, 'nota': 'nobis quaerat similique quisquam repellat unde possimus perferendis a ipsam cumque'}

{'fecha': '1971-02-07 19:52:50', 'agencia': 'Pando', 'monto': 321466.39, 'descripcion': 'Transferencia de fondos de ahorros', 'saldo': 3432321.95, 'nota': 'quibusdam mollitia deserunt'}

{'fecha': '1953-07-01 01:36:53', 'agencia': 'Chuquisaca', '

## Consulta 2: Fecha y agencia

Se realizará una búsqueda sobre los campos fecha y agencia. Siendo los parámetros de entrada una fecha inicial, una fecha final y una agencia.

In [71]:
start_date = "2019-01-31" 
end_date = "2019-12-31"
agencia = "Cochabamba"

start_time = time.time()
results = busqueda_fecha_agencia(start_date, end_date, agencia)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '2019-12-20 14:55:25', 'agencia': 'Cochabamba', 'monto': 71924.54, 'descripcion': 'Comisión de mantenimiento', 'saldo': 7171625.59, 'nota': 'magni aut nam provident'}

{'fecha': '2019-12-16 00:04:32', 'agencia': 'Cochabamba', 'monto': 273700.97, 'descripcion': 'Pago recibo de gas', 'saldo': 4218202.71, 'nota': 'rerum molestias obcaecati dolorum eveniet repellendus hic praesentium odit autem alias temporibus'}

{'fecha': '2019-07-04 09:30:37', 'agencia': 'Cochabamba', 'monto': 267080.69, 'descripcion': 'Ingreso en efectivo', 'saldo': 3530247.67, 'nota': 'nulla rerum explicabo unde qui iure ipsum consectetur saepe minus amet dicta recusandae'}

{'fecha': '2019-11-15 13:33:54', 'agencia': 'Cochabamba', 'monto': 370397.39, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7569939.65, 'nota': 'delectus a consequatur voluptatibus alias et'}

{'fecha': '2019-02-12 07:37:22', 'agencia': 'Cochabamba', 'monto': 362102.95, 'descripcion': 'Pago en interés', 'saldo': 885465.53, 'nota': 'a s

## Consulta 3: Rango de saldo, dos opciones de agencias y descripción

Se realizará una búsqueda sobre los campos saldo, agencia (OR 2 opciones) y descripción. Los parámetros de entrada son un rango de saldos, dos opciones de agencias, y una descripción.

In [72]:
saldo_min = 162342
saldo_max = 382304513
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = "Retiro en efectivo"

start_time = time.time()
results = busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '1977-04-02 05:06:15', 'agencia': 'La Paz', 'monto': 261529.35, 'descripcion': 'Retiro en efectivo', 'saldo': 8216476.33, 'nota': 'perspiciatis deleniti totam fugiat numquam iusto delectus adipisci rem dicta culpa pariatur'}

{'fecha': '1981-12-31 05:23:17', 'agencia': 'La Paz', 'monto': 256193.53, 'descripcion': 'Retiro en efectivo', 'saldo': 1631547.99, 'nota': 'enim unde quas esse'}

{'fecha': '1950-09-10 01:01:32', 'agencia': 'La Paz', 'monto': 364078.38, 'descripcion': 'Retiro en efectivo', 'saldo': 6036705.51, 'nota': 'earum assumenda sit veritatis dolores molestiae deserunt qui optio tenetur placeat iure'}

{'fecha': '2000-08-15 18:56:38', 'agencia': 'La Paz', 'monto': 299981.02, 'descripcion': 'Retiro en efectivo', 'saldo': 4785593.87, 'nota': 'facere rerum quisquam quas dolores'}

{'fecha': '1957-04-29 23:01:59', 'agencia': 'La Paz', 'monto': 179877.78, 'descripcion': 'Retiro en efectivo', 'saldo': 9320786.52, 'nota': 'dolorum sed natus eaque deserunt esse totam iust

## Agregación 1: Suma de saldos de una agencia

Se calculará la suma de los saldos de los datos agrupados de una agencia en específico.

In [73]:
agencia = "Beni"

start_time = time.time()
results = suma_saldo_agencia(agencia)
end_time = time.time()


print("="*100)
print(results['hits'])
print("="*100)
print(results['aggregations'])
print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 2.0806541, 'hits': [{'_index': 'extracto_cuenta_30m', '_id': 'yprYY4QBGbS6fiSm7uVJ', '_score': 2.0806541, '_source': {'fecha': '1955-06-10 17:59:28', 'agencia': 'Beni', 'monto': 182999.8, 'descripcion': 'Retención IVA', 'saldo': 365443.46, 'nota': 'quod voluptate quidem eius corporis blanditiis eveniet voluptatibus mollitia aut'}}, {'_index': 'extracto_cuenta_30m', '_id': '0JrYY4QBGbS6fiSm7uVJ', '_score': 2.0806541, '_source': {'fecha': '1991-04-05 09:34:58', 'agencia': 'Beni', 'monto': 111452.95, 'descripcion': 'Reintegro cajero automático', 'saldo': 530080.54, 'nota': 'repellat nihil ducimus iure voluptatibus mollitia non consequuntur quaerat ullam harum quas sint'}}, {'_index': 'extracto_cuenta_30m', '_id': '0ZrYY4QBGbS6fiSm7uVJ', '_score': 2.0806541, '_source': {'fecha': '2005-04-27 03:36:56', 'agencia': 'Beni', 'monto': 324726.77, 'descripcion': 'Pago en línea - Servicio de internet', 'saldo': 6662995.19, 'nota': 'nisi vo

## Agregación 2: Promedio de montos por agencia

In [74]:
start_time = time.time()
results = promedio_montos_agencia()
end_time = time.time()


print("="*100)
print(results['hits'])
print("="*100)
print(results['aggregations'])
print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'extracto_cuenta_30m', '_id': 'yJrYY4QBGbS6fiSm7uVJ', '_score': 1.0, '_source': {'fecha': '1985-01-08 00:43:31', 'agencia': 'Chuquisaca', 'monto': 224529.15, 'descripcion': 'Ingreso en efectivo', 'saldo': 3536179.16, 'nota': 'distinctio iste'}}, {'_index': 'extracto_cuenta_30m', '_id': 'yZrYY4QBGbS6fiSm7uVJ', '_score': 1.0, '_source': {'fecha': '1972-10-10 07:50:07', 'agencia': 'Oruro', 'monto': 314217.46, 'descripcion': 'Pago de nómina', 'saldo': 7217329.69, 'nota': 'quidem'}}, {'_index': 'extracto_cuenta_30m', '_id': 'yprYY4QBGbS6fiSm7uVJ', '_score': 1.0, '_source': {'fecha': '1955-06-10 17:59:28', 'agencia': 'Beni', 'monto': 182999.8, 'descripcion': 'Retención IVA', 'saldo': 365443.46, 'nota': 'quod voluptate quidem eius corporis blanditiis eveniet voluptatibus mollitia aut'}}, {'_index': 'extracto_cuenta_30m', '_id': 'y5rYY4QBGbS6fiSm7uVJ', '_score': 1.0, '_source': {'fecha': '1957-02-14 06:52:33',

# Consultas a la base de datos con 10 millones de registros y solo tres campos indexados (fecha, descripcion, nota)

En esta sección se revisará la operación de búsqueda para la colección de 10 millones de documentos indexados en solo tres campos: fecha, descripcion y nota.

In [75]:
idx = "extracto_cuenta_10m_3idx"

## Consulta 1: Todos los datos

Se realizará una búsqueda sobre todos los campos en la colección

In [76]:
query = {
    "match_all" : {}
}

start_time = time.time()
results = es.search(index=idx, query=query)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '1992-12-31 10:32:56', 'agencia': 'Chuquisaca', 'monto': 20373.61, 'descripcion': 'Transferencia de fondos de ahorros', 'saldo': 2177795.59, 'nota': 'a error veritatis quidem molestias aut ipsum laborum eveniet facilis'}

{'fecha': '2017-04-09 09:21:09', 'agencia': 'Cochabamba', 'monto': 366131.71, 'descripcion': 'Compra en comercio', 'saldo': 1426094.89, 'nota': 'sunt totam ea quos deleniti delectus odio at quibusdam quasi quisquam'}

{'fecha': '2001-10-22 20:50:54', 'agencia': 'Oruro', 'monto': 285350.58, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7751277.84, 'nota': 'at maxime'}

{'fecha': '1976-10-13 11:44:15', 'agencia': 'Oruro', 'monto': 93555.27, 'descripcion': 'Compra en comercio', 'saldo': 9441032.66, 'nota': 'obcaecati illo quod expedita esse tempore'}

{'fecha': '2017-12-16 03:00:57', 'agencia': 'Oruro', 'monto': 341414.1, 'descripcion': 'Pago recibo de agua', 'saldo': 5223453.17, 'nota': 'illo tempora fugit'}

{'fecha': '1990-11-09 01:41:06', 'agencia': 'La P

## Consulta 2: Fecha y agencia

Se realizará una búsqueda sobre los campos fecha y agencia. Siendo los parámetros de entrada una fecha inicial, una fecha final y una agencia.

In [77]:
start_date = "2019-01-31" 
end_date = "2019-12-31"
agencia = "Cochabamba"

start_time = time.time()
results = busqueda_fecha_agencia(start_date, end_date, agencia)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '2019-08-30 14:09:59', 'agencia': 'Cochabamba', 'monto': 102664.52, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 8005956.07, 'nota': 'minus'}

{'fecha': '2019-04-03 01:47:46', 'agencia': 'Cochabamba', 'monto': 95827.08, 'descripcion': 'Consignación en efectivo', 'saldo': 9466725.8, 'nota': 'voluptas voluptate velit nostrum deleniti molestias soluta'}

{'fecha': '2019-02-22 21:08:11', 'agencia': 'Cochabamba', 'monto': 288747.54, 'descripcion': 'Pago en interés', 'saldo': 2030500.88, 'nota': 'aperiam voluptatum qui unde dolor dicta reprehenderit commodi et exercitationem velit in'}

{'fecha': '2019-03-31 00:58:24', 'agencia': 'Cochabamba', 'monto': 221735.3, 'descripcion': 'Retiro en efectivo', 'saldo': 2680484.95, 'nota': 'quis quibusdam repudiandae minus vero'}

{'fecha': '2019-11-30 03:41:23', 'agencia': 'Cochabamba', 'monto': 367487.74, 'descripcion': 'Pago de nómina', 'saldo': 2329827.82, 'nota': 'asperiores sint vitae'}

{'fecha': '2019-12-31 14:07:27', 'agencia': 'Coc

## Consulta 3: Rango de saldo, dos opciones de agencias y descripción

Se realizará una búsqueda sobre los campos saldo, agencia (OR 2 opciones) y descripción. Los parámetros de entrada son un rango de saldos, dos opciones de agencias, y una descripción.

In [78]:
saldo_min = 162342
saldo_max = 382304513
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = "Retiro en efectivo"

start_time = time.time()
results = busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '1986-09-10 08:42:58', 'agencia': 'Santa Cruz', 'monto': 166347.14, 'descripcion': 'Retiro en efectivo', 'saldo': 3814290.17, 'nota': 'aperiam placeat fugit ex officiis minima amet doloremque iusto'}

{'fecha': '1984-05-21 23:48:38', 'agencia': 'Santa Cruz', 'monto': 222373.33, 'descripcion': 'Retiro en efectivo', 'saldo': 4525272.74, 'nota': 'eligendi esse voluptatibus'}

{'fecha': '1966-01-30 00:22:43', 'agencia': 'La Paz', 'monto': 306101.64, 'descripcion': 'Retiro en efectivo', 'saldo': 7153563.3, 'nota': 'itaque nulla hic saepe'}

{'fecha': '1977-04-03 23:37:39', 'agencia': 'La Paz', 'monto': 295277.2, 'descripcion': 'Retiro en efectivo', 'saldo': 6524128.2, 'nota': 'sed enim voluptatem veritatis reiciendis labore delectus optio'}

{'fecha': '2018-11-01 08:30:31', 'agencia': 'La Paz', 'monto': 282528.98, 'descripcion': 'Retiro en efectivo', 'saldo': 302616.62, 'nota': 'eos cum sequi labore omnis'}

{'fecha': '1970-06-07 03:05:26', 'agencia': 'Santa Cruz', 'monto': 272060

## Agregación 1: Suma de saldos de una agencia

Se calculará la suma de los saldos de los datos agrupados de una agencia en específico.

In [79]:
agencia = "Beni"

start_time = time.time()
results = suma_saldo_agencia(agencia)
end_time = time.time()


print("="*100)
print(results['hits'])
print("="*100)
print(results['aggregations'])
print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'extracto_cuenta_10m_3idx', '_id': '3n84aIQBPBNG2LESaO-h', '_score': 1.0, '_source': {'fecha': '1994-07-01 23:01:42', 'agencia': 'Beni', 'monto': 210236.11, 'descripcion': 'Comisión de mantenimiento', 'saldo': 8603958.75, 'nota': 'cupiditate impedit obcaecati hic'}}, {'_index': 'extracto_cuenta_10m_3idx', '_id': '5X84aIQBPBNG2LESaO-h', '_score': 1.0, '_source': {'fecha': '2006-11-05 20:49:20', 'agencia': 'Beni', 'monto': 263442.72, 'descripcion': 'Aporte inversor', 'saldo': 6956212.74, 'nota': 'vitae maiores minus'}}, {'_index': 'extracto_cuenta_10m_3idx', '_id': '7n84aIQBPBNG2LESaO-h', '_score': 1.0, '_source': {'fecha': '1960-12-23 20:55:39', 'agencia': 'Beni', 'monto': 318843.81, 'descripcion': 'Apertura de cuenta corriente', 'saldo': 4254979.3, 'nota': 'excepturi aspernatur ut non sit eos obcaecati'}}, {'_index': 'extracto_cuenta_10m_3idx', '_id': '9384aIQBPBNG2LESaO-h', '_score': 1.0, '_source': {

## Agregación 2: Promedio de montos por agencia

In [80]:
start_time = time.time()
results = promedio_montos_agencia()
end_time = time.time()


print("="*100)
print(results['hits'])
print("="*100)
print(results['aggregations'])
print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'extracto_cuenta_10m_3idx', '_id': '1X84aIQBPBNG2LESaO-h', '_score': 1.0, '_source': {'fecha': '1992-12-31 10:32:56', 'agencia': 'Chuquisaca', 'monto': 20373.61, 'descripcion': 'Transferencia de fondos de ahorros', 'saldo': 2177795.59, 'nota': 'a error veritatis quidem molestias aut ipsum laborum eveniet facilis'}}, {'_index': 'extracto_cuenta_10m_3idx', '_id': '1n84aIQBPBNG2LESaO-h', '_score': 1.0, '_source': {'fecha': '2017-04-09 09:21:09', 'agencia': 'Cochabamba', 'monto': 366131.71, 'descripcion': 'Compra en comercio', 'saldo': 1426094.89, 'nota': 'sunt totam ea quos deleniti delectus odio at quibusdam quasi quisquam'}}, {'_index': 'extracto_cuenta_10m_3idx', '_id': '1384aIQBPBNG2LESaO-h', '_score': 1.0, '_source': {'fecha': '2001-10-22 20:50:54', 'agencia': 'Oruro', 'monto': 285350.58, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7751277.84, 'nota': 'at maxime'}}, {'_index': 'extracto_cuenta_10

# Consultas a la base de datos con 30 millones de registros y solo tres campos indexados (fecha, descripcion, nota)

En esta sección se revisará la operación de búsqueda para la colección de 30 millones de documentos indexados en solo tres campos: fecha, descripcion y nota.

In [81]:
idx = "extracto_cuenta_30m_3idx"

## Consulta 1: Todos los datos

Se realizará una búsqueda sobre todos los campos en la colección

In [82]:
query = {
    "match_all" : {}
}

start_time = time.time()
results = es.search(index=idx, query=query)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '2006-07-21 08:28:08', 'agencia': 'Santa Cruz', 'monto': 196956.04, 'descripcion': 'Pago en línea - Servicio de internet', 'saldo': 381738.48, 'nota': 'obcaecati aspernatur ex harum doloribus doloremque'}

{'fecha': '1968-04-30 08:20:51', 'agencia': 'Tarija', 'monto': 9057.27, 'descripcion': 'Pago en interés', 'saldo': 1074640.58, 'nota': 'aliquam ipsum qui'}

{'fecha': '1994-07-26 17:51:03', 'agencia': 'Chuquisaca', 'monto': 206889.3, 'descripcion': 'Retiro en efectivo', 'saldo': 8857751.66, 'nota': 'consectetur fugiat voluptates'}

{'fecha': '1962-08-18 23:06:57', 'agencia': 'Tarija', 'monto': 114669.14, 'descripcion': 'Pago de cheque compensado', 'saldo': 5389451.89, 'nota': 'sequi eaque fugiat officia sit voluptatibus incidunt omnis nisi quasi'}

{'fecha': '1961-07-18 14:38:26', 'agencia': 'Santa Cruz', 'monto': 233935.71, 'descripcion': 'Pago recibo de gas', 'saldo': 6850822.73, 'nota': 'impedit'}

{'fecha': '1966-08-04 18:17:31', 'agencia': 'Cochabamba', 'monto': 390122

## Consulta 2: Fecha y agencia

Se realizará una búsqueda sobre los campos fecha y agencia. Siendo los parámetros de entrada una fecha inicial, una fecha final y una agencia.

In [83]:
start_date = "2019-01-31" 
end_date = "2019-12-31"
agencia = "Cochabamba"

start_time = time.time()
results = busqueda_fecha_agencia(start_date, end_date, agencia)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '2019-05-28 21:35:40', 'agencia': 'Cochabamba', 'monto': 92712.8, 'descripcion': 'Cobro seguro', 'saldo': 6730644.66, 'nota': 'impedit officiis quisquam ducimus minus nesciunt provident ipsa libero omnis doloremque eius sed'}

{'fecha': '2019-08-03 03:32:01', 'agencia': 'Cochabamba', 'monto': 71899.4, 'descripcion': 'Pago de servicio telefónico', 'saldo': 847438.81, 'nota': 'at architecto officia optio assumenda provident earum eligendi ducimus blanditiis voluptate'}

{'fecha': '2019-03-06 17:06:19', 'agencia': 'Cochabamba', 'monto': 397548.63, 'descripcion': 'Retención IVA', 'saldo': 7155522.09, 'nota': 'commodi'}

{'fecha': '2019-09-01 22:12:55', 'agencia': 'Cochabamba', 'monto': 205299.92, 'descripcion': 'Intereses', 'saldo': 6804152.77, 'nota': 'provident ex ducimus sunt optio commodi animi soluta'}

{'fecha': '2019-11-13 05:53:06', 'agencia': 'Cochabamba', 'monto': 380625.9, 'descripcion': 'Pago recibo de gas', 'saldo': 7405157.64, 'nota': 'officiis possimus tenetur temp

## Consulta 3: Rango de saldo, dos opciones de agencias y descripción

Se realizará una búsqueda sobre los campos saldo, agencia (OR 2 opciones) y descripción. Los parámetros de entrada son un rango de saldos, dos opciones de agencias, y una descripción.

In [85]:
saldo_min = 162342
saldo_max = 382304513
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = "Retiro en efectivo"

start_time = time.time()
results = busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'fecha': '1965-01-31 14:44:42', 'agencia': 'Santa Cruz', 'monto': 355469.84, 'descripcion': 'Retiro en efectivo', 'saldo': 569583.87, 'nota': 'ea quisquam aut ab minus sunt velit quos suscipit'}

{'fecha': '1971-05-07 11:43:24', 'agencia': 'Santa Cruz', 'monto': 196721.9, 'descripcion': 'Retiro en efectivo', 'saldo': 9712315.3, 'nota': 'vitae quibusdam quam fugit ipsum aliquid molestias nihil doloremque ipsam sed'}

{'fecha': '1999-12-03 08:17:14', 'agencia': 'Santa Cruz', 'monto': 167380.92, 'descripcion': 'Retiro en efectivo', 'saldo': 7219208.5, 'nota': 'sequi vero'}

{'fecha': '2014-02-26 04:05:12', 'agencia': 'Santa Cruz', 'monto': 315228.4, 'descripcion': 'Retiro en efectivo', 'saldo': 5602970.43, 'nota': 'error eos quasi ab iste reprehenderit quam eligendi ut illo recusandae'}

{'fecha': '2000-10-07 19:00:56', 'agencia': 'La Paz', 'monto': 330782.68, 'descripcion': 'Retiro en efectivo', 'saldo': 7151659.31, 'nota': 'ratione nemo'}

{'fecha': '2018-04-07 06:07:17', 'agencia': 'L

## Agregación 1: Suma de saldos de una agencia

Se calculará la suma de los saldos de los datos agrupados de una agencia en específico.

In [86]:
agencia = "Beni"

start_time = time.time()
results = suma_saldo_agencia(agencia)
end_time = time.time()


print("="*100)
print(results['hits'])
print("="*100)
print(results['aggregations'])
print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'extracto_cuenta_30m_3idx', '_id': 'Z769Z4QBPBNG2LESI3n6', '_score': 1.0, '_source': {'fecha': '1962-03-13 17:09:36', 'agencia': 'Beni', 'monto': 150308.02, 'descripcion': 'Debito Cta por ACH', 'saldo': 5049151.8, 'nota': 'error deserunt tenetur optio doloremque'}}, {'_index': 'extracto_cuenta_30m_3idx', '_id': 'cb69Z4QBPBNG2LESI3n6', '_score': 1.0, '_source': {'fecha': '2007-02-16 00:37:35', 'agencia': 'Beni', 'monto': 268940.12, 'descripcion': 'Pago recibo de luz', 'saldo': 6445665.08, 'nota': 'voluptatem velit aliquid magnam rerum'}}, {'_index': 'extracto_cuenta_30m_3idx', '_id': 'f769Z4QBPBNG2LESI3n6', '_score': 1.0, '_source': {'fecha': '1963-03-09 17:40:34', 'agencia': 'Beni', 'monto': 323094.79, 'descripcion': 'Deposito a cuenta', 'saldo': 6366524.78, 'nota': 'odio cumque necessitatibus officiis quam quae culpa deleniti'}}, {'_index': 'extracto_cuenta_30m_3idx', '_id': 'kL69Z4QBPBNG2LESI3n6', '_

## Agregación 2: Promedio de montos por agencia

In [87]:
start_time = time.time()
results = promedio_montos_agencia()
end_time = time.time()


print("="*100)
print(results['hits'])
print("="*100)
print(results['aggregations'])
print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'extracto_cuenta_30m_3idx', '_id': 'Vb69Z4QBPBNG2LESI3n6', '_score': 1.0, '_source': {'fecha': '2006-07-21 08:28:08', 'agencia': 'Santa Cruz', 'monto': 196956.04, 'descripcion': 'Pago en línea - Servicio de internet', 'saldo': 381738.48, 'nota': 'obcaecati aspernatur ex harum doloribus doloremque'}}, {'_index': 'extracto_cuenta_30m_3idx', '_id': 'Vr69Z4QBPBNG2LESI3n6', '_score': 1.0, '_source': {'fecha': '1968-04-30 08:20:51', 'agencia': 'Tarija', 'monto': 9057.27, 'descripcion': 'Pago en interés', 'saldo': 1074640.58, 'nota': 'aliquam ipsum qui'}}, {'_index': 'extracto_cuenta_30m_3idx', '_id': 'V769Z4QBPBNG2LESI3n6', '_score': 1.0, '_source': {'fecha': '1994-07-26 17:51:03', 'agencia': 'Chuquisaca', 'monto': 206889.3, 'descripcion': 'Retiro en efectivo', 'saldo': 8857751.66, 'nota': 'consectetur fugiat voluptates'}}, {'_index': 'extracto_cuenta_30m_3idx', '_id': 'WL69Z4QBPBNG2LESI3n6', '_score': 1.0, 